## 감정 분석 소개 

감정분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(sentiment) 수치를 계산하는 방법을 이용합니다. 그리고 이런 감성을 분류하는데는 지도학습 방법과 비지도 학습 방법이 사용됩니다. 
* 지도학습은 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터와 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일합니다. 
* 비지도 학습은 "lexicon"이라는 일종의 감성 어휘 사전을 이용합니다. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 잇으며 이를 이용해 문서의 긍정적 부정적 감성 여부를 판단합니다.

## 지도학습 기반 감성 분석 실습 -IMDB 영화평 

지도학습 감성분석을 해보기 위해서 우선 데이터를 kaggle에서 가져오도록 하겠습니다.

In [30]:
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [31]:
from sklearn.datasets import load_files
import pandas as pd 

review_df=pd.read_csv("./labeledTrainData.tsv",header=0,sep="\t",quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


로드된 데이터를 살펴보겠습니다. 
* id : 각 데이터의 id 
* sentiment : 영화평의 sentiment 결과 값, 1은 긍정적인 평가, 0은 부정적인 평가를 의미합니다 
* review : 영화평의 텍스트입니다.

In [32]:
(review_df.loc[0,"review"])

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally


html 형식이여서 그런지 "<br \>" 태그가 여전히 존재합니다 해당 문자열은 피처로 만들필요가 없기 때문에 해당 태그를 모두 공백처리로 바꾸도록 하겠습니다

In [9]:
import re 
# <br> html 태그는 replace 함수로 공백 변환 

review_df["review"]=review_df['review'].apply(lambda x: re.sub("<br />","",x))

print(review_df.loc[0,"review"])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [33]:
# 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review']=review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]"," ",x))

결정 값 클래스인 sentiment 칼럼을 별도로 추출해 결정 값 데이터 세트를 만들고, 원본 데이터 세트에서 id와 sentiment 칼럼을 삭제해 피처 데이터 세트를 생성합니다. 그리고 train_test_split으로 학습용/ 테스트용 데이터 셋으로 분리하겠습니다.

In [37]:
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df[["review"]]
print(class_df.shape,feature_df.shape)
X_train,X_test,y_train,y_test=train_test_split(feature_df,class_df,test_size=0.3,random_state=156)

print(X_train.shape,X_test.shape)

(25000,) (25000, 1)
(17500, 1) (7500, 1)


먼저 Count 벡터화를 적용해 예측 성능을 측정하고 다음으로 tf-idf 벡터화를 적용해보겠습니다. classifier에는 Logistic Regression 을 이용하고 이진 분류임을 고려해 테스트 데이터 세트의 정확도와 ROC-AUC를 모두 측정하겠습니다.

In [40]:
%%time

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱워드는 english, filtering, ngram은 (1,2)로 설정해 countVectorization 수행 
# LogisticRegression C는 10으로 설정 
pipeline=Pipeline([("cnt_vect",CountVectorizer(stop_words="english",ngram_range=(1,2))),("lr_clf",LogisticRegression(C=10))])

#Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행 predict_proba()는 roc_auc 때문에 수행 
pipeline.fit(X_train["review"],y_train)
prediction=pipeline.predict(X_test["review"])
preb_prob=pipeline.predict_proba(X_test["review"])[:,1]

print(f"예측 정확도는 : {accuracy_score(y_test,prediction):.4f}, ROC-AUC는 {roc_auc_score(y_test,preb_prob):.4f}")


C:\Users\sunkeun_jo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


예측 정확도는 : 0.8865, ROC-AUC는 0.9506
Wall time: 43.9 s


In [41]:
%%time 

pipeline=Pipeline([("tfidf_vect",TfidfVectorizer(stop_words="english",ngram_range=(1,2))),("lr_clf",LogisticRegression(C=10))])
pipeline.fit(X_train["review"],y_train)
pred=pipeline.predict(X_test["review"])
pred_prob=pipeline.predict_proba(X_test["review"])[:,1]

print(f"예측 정확도는 : {accuracy_score(y_test,pred):.4f}, ROC-AUC는 {roc_auc_score(y_test,pred_prob):.4f}")

예측 정확도는 : 0.8865, ROC-AUC는 0.9506
Wall time: 29.9 s


In [42]:
print(f"예측 정확도는 : {accuracy_score(y_test,pred):.4f}, ROC-AUC는 {roc_auc_score(y_test,pred_prob):.4f}")

예측 정확도는 : 0.8932, ROC-AUC는 0.9600


tf-idf 기반 피처 벡터화의 예측 성능이 소폭 상승했습니다!